# Mask Detection

In [5]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [3]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [4]:
TRAINING_DIR = "C:/Users/Dell/Desktop/study/ML codes/Mask detection/Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "C:/Users/Dell/Desktop/study/ML codes/Mask detection/Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [6]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')


In [7]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
132/132 [==============================] - 76s 573ms/step - loss: 0.7086 - acc: 0.6849 - val_loss: 0.3376 - val_acc: 0.8918
Epoch 2/10
132/132 [==============================] - 69s 521ms/step - loss: 0.3752 - acc: 0.8516 - val_loss: 0.0162 - val_acc: 0.9536
Epoch 3/10
132/132 [==============================] - 69s 523ms/step - loss: 0.3103 - acc: 0.8798 - val_loss: 0.1734 - val_acc: 0.9691
Epoch 4/10
132/132 [==============================] - 68s 518ms/step - loss: 0.2882 - acc: 0.9003 - val_loss: 0.3619 - val_acc: 0.9742
Epoch 5/10
132/132 [==============================] - 72s 545ms/step - loss: 0.2641 - acc: 0.9011 - val_loss: 0.0701 - val_acc: 0.9897
Epoch 6/10
132/132 [==============================] - 72s 546ms/step - loss: 0.2299 - acc: 0.9094 - val_loss: 0.0209 - val_acc: 0.9742
Epoch 7/10
132/132 [==============================] - 71s 541ms/step - loss: 0.1956 - acc: 0.9262 - val_loss: 0.3497 - val_acc: 0.9691
Epoch 8/10
132/132 [==============================] - 7

In [1]:
model.save_weights("model2-009.model")


NameError: name 'model' is not defined

In [3]:
import cv2
import numpy as np
from keras.models import load_model
model=load_model("model2-009.model")
results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}
rect_size = 4
cap = cv2.VideoCapture(0) 
haarcascade = cv2.CascadeClassifier('C:/Users/Dell/Desktop/study/ML codes/Face reco/face recon 3.0/haarcascade_frontalface_default.xml')


In [4]:

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 
    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break
cap.release()
cv2.destroyAllWindows()